In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import Counter
import plotly.express as px 


In [ ]:
data = pd.read_csv("../input/customer-analytics/Train.csv")

In [ ]:
data.head()

In [ ]:
Counter(data['Reached.on.Time_Y.N'])

In [ ]:
data['Reached.on.Time_Y.N'] = data['Reached.on.Time_Y.N'].map({0:'Yes',1:'No'})

In [ ]:
data.head()

In [ ]:
Counter(data['Reached.on.Time_Y.N'])

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

#### Checking for Balanced Data set

In [ ]:
data.groupby(['Reached.on.Time_Y.N'])['Reached.on.Time_Y.N'].count().plot.bar()

In [ ]:
data.head()

In [ ]:
sns.catplot(x='Mode_of_Shipment',y='Cost_of_the_Product',hue='Gender',col='Reached.on.Time_Y.N',data=data,kind='bar')

In [ ]:
sns.catplot(x='Mode_of_Shipment',hue='Gender',col='Reached.on.Time_Y.N',data=data,kind='count')

In [ ]:
data.head()

In [ ]:
sns.catplot(x='Warehouse_block',hue='Gender',col='Reached.on.Time_Y.N',data=data,kind='count')

In [ ]:
sns.catplot(x='Mode_of_Shipment',col='Reached.on.Time_Y.N',data=data,kind='count')

In [ ]:
reached = data['Reached.on.Time_Y.N'].value_counts().reset_index()
reached.columns = ['Time','Values']
px.pie(reached,names='Time',values='Values',color_discrete_sequence=px.colors.sequential.Darkmint_r)

In [ ]:
sns.distplot(data['Cost_of_the_Product'],kde=True)

In [ ]:
xs=list(data['Mode_of_Shipment'].unique())
print(xs)

In [ ]:
data.groupby(['Mode_of_Shipment'])['Weight_in_gms'].count().plot.bar()

In [ ]:
data.head()

In [ ]:
plt.figure(figsize=(40,20))
sns.pairplot(data=data,hue='Reached.on.Time_Y.N')
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(data.corr(),annot=True,vmin=0,vmax=1,linewidths=3,linecolor='blue')

In [ ]:
feat = set()
corr_data = data.corr()
for i in range(len(corr_data.columns)):
    for j in range(i):
        if corr_data.iloc[i,j] >= 0.05:
            colname = corr_data.columns[i]
            feat.add(colname)
print(feat)

In [ ]:
data.head()

In [ ]:
data['Reached.on.Time_Y.N'] = data['Reached.on.Time_Y.N'].map({'Yes':0,'No':1})

In [ ]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [ ]:
X = pd.get_dummies(X,drop_first=True)

In [ ]:
X.head()

In [ ]:
LE = StandardScaler()


In [ ]:
X = pd.DataFrame(LE.fit_transform(X),columns=X.columns)

In [ ]:
X.head()

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mutual_info = mutual_info_classif(X,y)
mutal_data = pd.Series(mutual_info, index = X.columns)
mutal_data.sort_values(ascending=True)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
ranked_feat = pd.Series(model.feature_importances_,index=X.columns)
ranked_feat.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
X.drop(columns=['ID'],axis=1,inplace=True)

In [ ]:
XX_train,X_test,yy_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_cv,y_train,y_cv = train_test_split(XX_train,yy_train,test_size=0.2,random_state=42)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RF = RandomForestClassifier()
RF_param = {'n_estimators':range(1,500,10), 'max_depth':range(1,50)}
RF_search = RandomizedSearchCV(RF,RF_param,scoring='neg_mean_squared_error',cv=6,n_jobs=-1)
RF_search.fit(X_cv,y_cv)

In [ ]:
print(RF_search.best_params_)
print(RF_search.best_score_)
print(RF_search.best_estimator_)


In [ ]:
RF_result = RandomForestClassifier(n_estimators=11,max_depth=15)
RF_result.fit(X_train,y_train)
pred1 = RF_result.predict(X_test)

In [ ]:
Acc1 = accuracy_score(y_test,pred1)

In [ ]:
print(Acc1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
KN = KNeighborsClassifier()
param_grid = {'n_neighbors':range(1,20)}
grid = GridSearchCV(KN,param_grid,cv=10,scoring='neg_mean_squared_error')

In [ ]:
grid.fit(X_cv,y_cv)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)

In [ ]:
KNN = KNeighborsClassifier(n_neighbors=13)
KNN.fit(X_train,y_train)

In [ ]:
pred2 = KNN.predict(X_test)
Acc2 = accuracy_score(y_test,pred2)
print(accuracy_score(y_test,pred2))

In [ ]:
param_deci = {'criterion':['gini','entropy'],
             'max_depth':range(1,10),
             }

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
grid_s = GridSearchCV(DT,param_deci,cv=10,n_jobs=-1)
grid_s.fit(X_cv,y_cv)

In [ ]:
print(grid_s.best_params_)
print(grid_s.best_score_)
print(grid_s.best_estimator_)

In [ ]:
DTT = DecisionTreeClassifier(max_depth=3,criterion='gini')

In [ ]:
DTT.fit(X_train,y_train)

In [ ]:
pred3=DTT.predict(X_test)

In [ ]:
Acc3=accuracy_score(y_test,pred3)
print(accuracy_score(y_test,pred3))

In [ ]:
Result = pd.DataFrame({'Algo':['RandomForest','KNN','DecisionTree'],
                      'Accuracy':[Acc1,Acc2,Acc3]})

In [ ]:
Result

In [ ]:
sns.catplot(x='Algo',y='Accuracy',data=Result,kind='bar')